In [1]:
# pip install kaggle if you're in a non-Kaggle env
# pip install simpletransformers if you have internet

In [2]:
! head -n 5 /kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv

worker,less_toxic,more_toxic
313," This article sucks 

woo woo wooooooo","WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!!!!!!!!??????????????????????????????????????????????????????!!!!!!=
WHER IS YOUR SEXY PIC GONE FROM YOUR MAIN PAGE???????? put it back"


In [3]:
import pandas as pd
trainer = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv")

toxic = trainer[["more_toxic"]]
toxic["text"] = toxic["more_toxic"]
toxic["label"] = 0.75

nontoxic = trainer[["less_toxic"]]
nontoxic["text"] = nontoxic["less_toxic"]
nontoxic["label"] = 0.35

combined = toxic.append(nontoxic).sample(frac=1, random_state=101)
combined = combined[["text", "label"]]
combined["label"] = combined["label"].astype('float')
combined.head(2)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,text,label
27323,Wha's a reliable source mike? Something that c...,0.35
10783,Your contempt is noted. I hope I don't dissapp...,0.35


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDRegressor

In [5]:
sk_reg = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                   ('reg', SGDRegressor()),
])

In [6]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(combined, random_state=101, train_size=0.5)

sk_reg = sk_reg.fit(train['text'], train['label'])

In [7]:
! head -n 5 /kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv

comment_id,text
114890,"""
 

Gjalexei, you asked about whether there is an """"anti-editorializing"""" policy here.  There is, and it's called wikipedia:neutral point of view.  It discusses at some length  the case of what we should do when writing about a subject which most of us find repugnant.  Whilst you're not likely to get too many defenders of FGM here, the need for the policy should be clearer for articles like abortion, for instance.


In [8]:
import csv

texts = []
ids = []
with open('/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv', 'r') as rfile:
  rdr = csv.reader(rfile)
  batch = None
  for row in rdr:
    if batch is None:
        batch = True
        continue
    ids.append(row[0])
    texts.append(row[1].replace('"', '').strip())

In [9]:
! head -n 5 /kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv

comment_id,score
114890,0.5
732895,0.5
1139051,0.5
1434512,0.5


In [10]:
texts[0]

"Gjalexei, you asked about whether there is an anti-editorializing policy here.  There is, and it's called wikipedia:neutral point of view.  It discusses at some length  the case of what we should do when writing about a subject which most of us find repugnant.  Whilst you're not likely to get too many defenders of FGM here, the need for the policy should be clearer for articles like abortion, for instance.\n\nIf something you write is edited and you're not sure why, please continue to question such edits on the talk page.  Sometimes, you'll learn more about wikipedia policy.  Sometimes, you'll find out that some other people working on here can get it flat-out wrong ) Robert Merkel"

In [11]:
%%capture
op = []
id_idx = 0

with open('submission.csv', 'w') as opfile:
  writer = csv.writer(opfile)
  writer.writerow(["comment_id","score"])

  labels = sk_reg.predict(texts)
  for label in labels:
      writer.writerow([
        ids[id_idx],
        float(label)
      ])
      id_idx += 1

In [12]:
! head -n 5 submission.csv